> Run Anaconde Prompt with Admin Rights

Get ipyleaflet help https://ipyleaflet.readthedocs.io/en/latest/index.html

Install "pip install openlocationcode"

Install "conda install gpxpy"

Get Coordinates from https://www.google.de/maps/@52.3097709,9.7330615,14z

GPS converter https://www.gpsvisualizer.com/

GeoJSON Daten http://opendatalab.de/projects/geojson-utilities/

GPX Editor Online https://www.gpxeditor.co.uk/map
GPX Tracks http://www.bikehike.co.uk/mapview.php

For GPS see http://research.ganse.org/datasci/gps/

GPS Viewer https://www.j-berkemeier.de/GPXViewer/

# Create base map for Hemmingen
Tipps: ScaleControl is not compatible with embed_html

In [250]:
# conda install -c conda-forge ipyleaflet 
## or
# conda config --add channels conda-forge
# conda install ipyleaflet 
from ipyleaflet import Map, basemaps, FullScreenControl, LayersControl, ScaleControl
from sidecar import Sidecar

def mapselection(i):
    switcher={
                0:basemaps.OpenStreetMap.Mapnik,
                1:basemaps.Esri.WorldImagery,
             }
    return switcher.get(i,"Invalid value of mapselection")

# center = (52.30, 9.73)
# center = (52.2995, 9.7255)
# zoom = 13
center = (52.26, 9.75)
zoom = 12
basemap = mapselection(1)

m = None
m = Map(basemap=basemap,center=center, zoom=zoom, scroll_wheel_zoom=True) # zoom_offset=??
m.layout.height='800px' 
m.layout.width='1013px'
# m.layout.width='700px' for 900 width

# add controls
m.add_control(FullScreenControl(position='topleft'))
m.add_control(LayersControl(position='topleft'))


m

Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [251]:
from ipyleaflet import basemap_to_tiles

satellit = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
m.add_layer(satellit)

# Load and show Borders

In [252]:
## Read all Border files
from os import listdir
from os.path import isfile, join

GrenzPath = 'Grenzen'

Grenzfiles = [join(GrenzPath,f) for f in listdir(GrenzPath) if isfile(join(GrenzPath, f))]
Grenzfiles

['Grenzen\\Hemmingen.gpx', 'Grenzen\\Pattensen.gpx']

In [253]:
# >conda install -c conda-forge gpxpy

from ipyleaflet import  Polyline
import gpxpy
import gpxpy.gpx
import re
from ipywidgets import HTML
from ipyleaflet import MarkerCluster

grenzen = []
for file in Grenzfiles:
    gpxfilefp = open(file, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
    gpx = gpxpy.parse(gpxfilefp)

    data = [[i.latitude, i.longitude] for i in gpx.tracks[0].segments[0].points]

    line = Polyline(
        name = 'Grenzen',
        locations=data,
        color="orange",
        opacity = 0.5,
        weight = 5,
        fill=False
    )

    match = re.search('\\\\(.+?)\.', file)
    if match:
        found = match.group(1)
    
    Poptext = HTML()
    Poptext.value = found
    line.popup = Poptext

    # m.add_layer(ant_path)

    grenzen.append(line)
    
GrenzenCluster = MarkerCluster(name = 'Grenzverläufe', markers= grenzen)
m.add_layer(GrenzenCluster);

m

Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

# ADFC Icon and FSW Location

In [254]:
from ipyleaflet import Icon, Marker

iconPos = (52.326298, 9.676757)
icon = Icon(icon_url='https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/ADFCLogo.png', icon_size=[120, 40])
markIcon = Marker(name='Icon',location=iconPos, icon=icon)
icon.icon_url

'https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/ADFCLogo.png'

In [255]:
from ipywidgets import HTML
from ipyleaflet import AwesomeIcon, Marker

locFSW = (52.319109, 9.723495)

# icon
iconGear = AwesomeIcon(
    name='gear',
    marker_color='blue',
    icon_color='darkred',
    spin=True
)

markerFSW = Marker( icon=iconGear, 
                    draggable= False,
                    location=locFSW,
                    title = 'klick mich')

# Popup Window
htmlFSW = HTML()
htmlFSW.value = '''<a href="http://adfc-hemmingen-pattensen.github.io/" 
                    target="_blank">ADFC Selbsthilfe-Werkstatt</a>'''
markerFSW.popup = htmlFSW

In [256]:
from ipywidgets import HTML
from ipyleaflet import MarkerCluster

ADFC_cluster = MarkerCluster(
    name = 'ADFC Info',
    markers=(markerFSW, markIcon)
)



m.add_layer(ADFC_cluster);
    
m;

# Verbingungswünsche

pass GPX file

In [257]:
## Read all Route file
from os import listdir
from os.path import isfile, join

WunschPath = 'Maengel'

wunschfiles = [join(WunschPath,f) for f in listdir(WunschPath) if isfile(join(WunschPath, f))]
print(len(wunschfiles))
wunschfiles[:5]

28


['Maengel\\7M9R+3Xtrk.gpx',
 'Maengel\\7PPR+3Ctrk.gpx',
 'Maengel\\7PVG+Q8trk.gpx',
 'Maengel\\8P3H+3Htrk.gpx',
 'Maengel\\8P5P+FRtrk.gpx']

In [258]:
# >conda install gpxpy

from ipyleaflet import  Polyline
import gpxpy
import gpxpy.gpx
import re

wuensche = []
for file in wunschfiles:
    gpxfilefp = open(file, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
    gpx = gpxpy.parse(gpxfilefp)

    data = [[i.latitude, i.longitude] for i in gpx.tracks[0].segments[0].points]

    line = Polyline(
        name = 'Wunschverbindungen',
        locations=data,
        color="red",
        weight =3,
        fill=False
    )

    match = re.search('\\\\(.+?)\.', file)
    if match:
        found = match.group(1)
    
    Poptext = HTML()
    Poptext.value = found
    line.popup = Poptext

    # m.add_layer(ant_path)

    wuensche.append(line)
WunschCluster = MarkerCluster(name = 'Wunschverbindungen', markers= wuensche)
m.add_layer(WunschCluster);

m;

# Load Excel as dataframe

In [259]:
import pandas as pd

df = pd.read_excel(r'Maengelliste.xlsx', sheet_name='Sheet1', skiprows=5, header=0)
df.head()
df.tail()

,PlusCode,GoogleMapLink,Titel,Einstelldatum,Informationsquelle,Ortsbeschreibung,Mangel,Maßnahmenvorschlag,Status,Erledigt,...,Route04,Route05,Route06,Route07,Route08,Route09,Route10,B3alt,Dorfstr.,Allg.
257,7PXM+GM,7PXM+GM,Defekter Belag / Löcher,2022-07-06,ADFC,Tal der Büffel,Der Belag ist in einem schlechten Zustand. Hat...,Ausbessern,offen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,8P8W+CH,8P8W+CH,Querrillen im Belag,2022-07-06,ADFC,Im Hammfeld,Der Belag ist in einem schlechten Zustand. Hat...,Ausbessern,offen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,8P6V+4C,8P6V+4C,Schotter Weg zu schmal,2022-07-06,ADFC,Vor Hundephulfsweg Brücke,Der Weg ist zu schmal und keine glatte Oberfläche,Asphaltieren,offen,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,8PFJ+PH,8PFJ+PH,Pictogramme auf südlicher Seite,2022-07-06,ADFC,Pictogramme auf südlicher Seite,Klare Kennzeichnung fehlt,Pictogramme,erledigt,x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,8PJR+26,8PJR+26,"Durchfahrtverbot für alle, auch Fahrräder.",2022-07-06,ADFC,Hohe Bünte - Nähe Freibad,Radfahrer frei fehlt,Radfahrer frei anbringen,erledigt,x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Decode Open Location Code
https://plus.codes/

In [260]:
from openlocationcode import openlocationcode as olc
import ipywidgets as widgets

In [261]:
css = """
<style>
h1 {
  font-weight: bold;
  color: #000;
  font-size: 14px;
}

h2 {
  font-weight: bold;
  color: #000;
  font-size: 13px;
}
h3 {
  font-weight: bold;
  color: #000;
  font-size: 12px;
}
</style>
"""

In [262]:
# generate Hash code
import hashlib

def generateHash(RawTitel):
    hash_object = hashlib.md5(RawTitel.encode())
    return hash_object.hexdigest()

#test
generateHash("Hello World")

'b10a8db164e0754105b7a99be72e3fe5'

In [263]:
import urllib.parse

def buildMarkdownText(df,i,routeName):
    plusCode = str(df.loc[i,'PlusCode'])
    RawTitel = str(df.loc[i,'Titel'])
    
    anchorcode = generateHash(RawTitel)
 
    anchor = '<a name="{}"></a>\n\n'.format(anchorcode)
    TextTitel = "# {}\n\n".format(RawTitel)
    if type(TextTitel) != str:
        TextTitel =  '*Kein Titel vorhanden*\n\n'
    
    BaseURL = "https://adfc-hemmingen-pattensen.github.io/MaengelKarte/index.html"
    GMapURL = "https://www.google.com/maps/search/?api=1&query=9F4F{}%2B{}".format(plusCode[:4],plusCode[-2:])
    TextPlusCode = "- Plus Code: [{}]({})\n".format(plusCode,GMapURL)

    #Mängelliste = '- Mängelliste: [Link]({})\n'.format(BaseURL+'#'+anchorcode)
    Mängelliste = '- Mängelliste: <a href={} target="_blank">Link</a>\n'.format(BaseURL+'#'+anchorcode)
      
    Einstelldatum = "- Einstelldatum: {}\n".format(str(df.loc[i,'Einstelldatum'])[:10])
    Informationsquelle = "- Informationsquelle: {}\n".format(str(df.loc[i,'Informationsquelle']))
    Betrifft = "- **Betrifft: {}**\n".format(routeName)
    Ortsbeschreibung = "\n### Ortsbeschreibung:\n\n {}\n".format(str(df.loc[i,'Ortsbeschreibung']))
    Mangel = "\n### Mangel:\n\n {}\n".format(str(df.loc[i,'Mangel']))
    Maßnahmenvorschlag = "\n### Maßnahmenvorschlag:\n\n {}\n".format(str(df.loc[i,'Maßnahmenvorschlag']))
    Status = "\n### Status:\n\n {}\n".format(str(df.loc[i,'Status']))
    hr = "\n---\n"
    
    return( anchor
           +TextTitel
           #+Betrifft
           +TextPlusCode
           +Mängelliste
           +Einstelldatum
           +Informationsquelle
           +Ortsbeschreibung
           +Mangel
           +Maßnahmenvorschlag
           +Status
           +hr
          ) 

In [264]:
import random
import string

def randomString(stringLength=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

In [265]:
### Create GPX Waypoint file <== New 2020-05-16-Sa JSp

def CreateGPXfile(gpx, gpxfilefp,coor,short, mdText):
    wpt = gpxpy.gpx.GPXWaypoint(latitude = coor.latitudeCenter,longitude=coor.longitudeCenter)
    wpt.description = mdText
    wpt.name = short+'_'+randomString(5)
    gpx.waypoints.append(wpt)
    return gpx

In [266]:
listOfRoutes = ['Route01','Route02','Route03','Route04','Route05','Route06','Route07','Route08','Route09','Route10','-']


In [267]:
# extract and convert 1st row  - openlocationcode 
# pip install openlocationcode
from ipyleaflet import  AntPath
from openlocationcode import openlocationcode as olc
import os.path
from markdown import markdown
import re

regexMarkdownImg = re.compile(r"!\[(.*?)\]\((.*?)\)", re.IGNORECASE) #https://www.regexpal.com/95855

AnzahlMarkers = len(df['PlusCode'])
print('Anzahl der Mängel {0}'.format(AnzahlMarkers))

allMDdescriptions = ""

fname = r"GPSOutput/maengel.gpx"
gpxoutfp = open(fname, 'w', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
gpx = gpxpy.gpx.GPX()


## Spezialfall einzelne Mängel #listOfRoutes = ['Mängel10']


# isRouteSum = False * len(listOfRoutes)
# loop over routes
# for routeName in listOfRoutes:
#     if routeName != '-':
#         isRoute = df[routeName].notnull()
#         isRouteSum |= isRoute
#         dfRoute = df[isRoute]
#         ClusterName = "Mängel{}".format(routeName[5:7])
#     else:
#         dfRoute = df[~isRouteSum]
#         ClusterName = 'Restliche Mängel'

markerMaengel = []
    #for i in dfRoute.index:
    
shortold = 0 # 2022-07-09-Sat

for i in range(0,AnzahlMarkers):

    # read OLC
#    short = str(dfRoute.loc[i,'PlusCode'])[:7]


    short = df.loc[i,'PlusCode']
    
    if short == shortold: #2022-07-09-Sat 
        shortcount +=1
    else:
        shortcount = 0
    
    print("short=",short, shortold, shortcount)
    shortold = short

    if olc.isValid(short): 
        
#        mdText = buildMarkdownText(dfRoute,i,routeName)
        mdText = buildMarkdownText(df,i,'xxxx')

        
        subMDimg = r'![\1](img\\\2)'
        
        # Achtung: Spezialfall für den 2020-06-24 - später wieder entfernen
        # if i <= 200:  # Nur Hemminger Mängel
        if i <= 300:  # Alle Mängel
            allMDdescriptions += regexMarkdownImg.sub(subMDimg, mdText)+"\n"
        
        # replace md img with html img and use width
        subHTMLimg = r'<img src="https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/\2" width="300">'
        mdTextPopUp = regexMarkdownImg.sub(subHTMLimg, mdText)
        
        htmlDescription = HTML()
        htmlDescription.value = css+'<div>'+markdown(mdTextPopUp)+'</div>'
        
        # convert OLC to Lat,Lon
        long = olc.recoverNearest(short, 52.30, 9.73)
        coor = olc.decode(long)
        pos =(coor.latitudeCenter, coor.longitudeCenter)
        
        # Wenn erledigt mach Icon Grün 2020-09-18-Fr
        done = str(df.loc[i,'Erledigt'])
        if done == "x":
            icon = Icon(icon_url='https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/pin_green.png', 
                        icon_size=[281/15,641/15], 
                        icon_anchor=[281/30,641/15])
        elif done == "v":
            icon = Icon(icon_url='https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/pin_yellow.png', 
                        icon_size=[281/15,641/15], 
                        icon_anchor=[281/30,641/15])
        elif done == "o":
            icon = Icon(icon_url='https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/pin_gray.png', 
                        icon_size=[281/15,641/15], 
                        icon_anchor=[281/30,641/15])
        else:
            icon = Icon(icon_url='https://adfc-hemmingen-pattensen.github.io/MaengelKarte/img/pin_red.png', 
                        icon_size=[281/15,641/15], 
                        icon_anchor=[281/30,641/15])
        
        markerMangel = Marker(title="klick",
                              location=pos, 
                              icon=icon, 
                              draggable=False, 
                              rotation_angle=20 + shortcount*30,  
                              rotation_origin='bottom center'
                              )
        
        markerMangel.popup = htmlDescription
        ## check https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html
        markerMangel.popup_min_width = 500
        markerMangel.popup_max_height = 500

        markerMaengel.append(markerMangel)
            
        ## Spezialfall einzelne Mängel - sonst auskommentieren
        ## Spezialfall ohne Cluster
        m.add_layer(markerMangel);
        
        gpx = CreateGPXfile(gpx, gpxfilefp,coor,short, mdText)
            
       
    else:
        print(i+7, end=" ")
    
    
#MaengelCluster = MarkerCluster(name = ClusterName, markers= markerMaengel)
MaengelCluster = MarkerCluster(name = 'Maengel', markers= markerMaengel)


## Spezialfall einzelne Mängel - bitte auskommentieren# 
## Spezialfall ohne Cluster
        
# m.add_layer(MaengelCluster);
   
          
f = open('maengel.md', 'w',encoding='utf8')
f.write(allMDdescriptions)
f.close()

gpxoutfp.write(gpx.to_xml())
gpxoutfp.close()
    
m

Anzahl der Mängel 262
short= 7M9R+3X 0 0
short= 7M9R+3X 7M9R+3X 1
short= 7MGH+C3 7M9R+3X 0
short= 7MMM+X7 7MGH+C3 0
short= 7MVW+C2 7MMM+X7 0
short= 7PP7+85 7MVW+C2 0
short= 7PPR+3C 7PP7+85 0
short= 7PVG+Q8 7PPR+3C 0
short= 7PXR+R8 7PVG+Q8 0
short= 7PXX+MW 7PXR+R8 0
short= 7QPF+GM 7PXX+MW 0
short= 8M7R+QH 7QPF+GM 0
short= 8P2R+R2 8M7R+QH 0
short= 8P2R+R2 8P2R+R2 1
short= 8P37+8H 8P2R+R2 0
short= 8P3H+3H 8P37+8H 0
short= 8P3H+3H 8P3H+3H 1
short= 8P3P+VP  8P3H+3H 0
24 short= 8P3Q+5M 8P3P+VP  0
short= 8P3Q+5P 8P3Q+5M 0
short= 8P3Q+R5 8P3Q+5P 0
short= 8P4P+CQ 8P3Q+R5 0
short= 8P4Q+9C 8P4P+CQ 0
short= 8P4X+V6 8P4Q+9C 0
short= 8P57+6C 8P4X+V6 0
short= 8P5J+MJ 8P57+6C 0
short= 8P5M+RH 8P5J+MJ 0
short= 8P5M+RH 8P5M+RH 1
short= 8P5M+RH 8P5M+RH 2
short= 8P5M+VG 8P5M+RH 0
short= 8P5M+VJ 8P5M+VG 0
short= 8P5P+FR 8P5M+VJ 0
short= 8P5V+VQ 8P5P+FR 0
short= 8P6H+GP 8P5V+VQ 0
short= 8P6J+HX 8P6H+GP 0
short= 8P6J+HX 8P6J+HX 1
short= 8P6J+HX 8P6J+HX 2
short= 8P6J+WP 8P6J+HX 0
short= 8P6M+95 8P6J+WP 0
shor

Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [268]:
## Read all Route file
from os import listdir
from os.path import isfile, join

RoutenPath = 'Routen'

onlyfiles = [join(RoutenPath,f) for f in listdir(RoutenPath) if isfile(join(RoutenPath, f))]

## Spezialfall einzelne Mängel 
# onlyfiles = ['Routen\\Route10.gpx']

onlyfiles

['Routen\\Route01.gpx',
 'Routen\\Route02.gpx',
 'Routen\\Route03.gpx',
 'Routen\\Route04.gpx',
 'Routen\\Route05.gpx',
 'Routen\\Route06.gpx',
 'Routen\\Route07.gpx',
 'Routen\\Route08.gpx',
 'Routen\\Route09.gpx',
 'Routen\\Route10.gpx']

In [269]:
from ipyleaflet import  AntPath
import gpxpy
import gpxpy.gpx

Routen = []
for file in onlyfiles:
    gpxfilefp = open(file, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
    gpx = gpxpy.parse(gpxfilefp)

    data = [[i.latitude, i.longitude] for i in gpx.tracks[0].segments[0].points]

    match = re.search('\\\\(.+?)\.', file)
    if match:
        found = match.group(1)
        
    ant_path = AntPath(
        name = found,
        title = found,
        locations=data,
        dash_array=[1, 10],
        delay=1000,
        color='#7590ba',
        pulse_color='#3f6fba'
    )

        
    Poptext = HTML()
    Poptext.value = found
    ant_path.popup = Poptext

    m.add_layer(ant_path)

m

Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [270]:
## Read all Route file
from os import listdir
from os.path import isfile, join

RoutenPath = 'Unfallorte'

onlyfiles = [join(RoutenPath,f) for f in listdir(RoutenPath) if isfile(join(RoutenPath, f))]

onlyfiles

['Unfallorte\\UnfallorteFahrrad2017.gpx',
 'Unfallorte\\UnfallorteFahrrad2018.gpx']

In [271]:
from ipyleaflet import  AntPath
import gpxpy
import gpxpy.gpx

iconUF = Icon(icon_url='img\\Unfall.png', icon_size=[12*3, 7*3])

Routen = []
for file in onlyfiles:
    gpxfilefp = open(file, 'r', encoding='utf-8-sig') #use sig if file has Byte Order Mark (BOM)
    gpx = gpxpy.parse(gpxfilefp)

    markerUnfälle = []
    for unfall in gpx.waypoints:
        pos =(unfall.latitude, unfall.longitude)
        
        markerUnfall = Marker(
                          location=pos, 
                          icon=iconUF, 
                          draggable=False, 
                          )
        
        htmlDescription = HTML()
        htmlDescription.value = 'Fahrradunfall {}<br> <a href="https://unfallatlas.statistikportal.de" target="_blank">https://unfallatlas.statistikportal.de</a>'.format(file[-8:-4])
        markerUnfall.popup = htmlDescription
        markerUnfälle.append(markerUnfall)
            
        UnfallCluster = MarkerCluster(name = file[11:-4], markers= markerUnfälle)


    # m.add_layer(UnfallCluster);
m


Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [272]:
from ipyleaflet import MeasureControl

measure = MeasureControl(
    position='topright',
    active_color = 'orange',
    primary_length_unit = 'kilometers',
    primary_area_unit = 'hectares'
)
m.add_control(measure)

measure.completed_color = 'red'


m

Map(center=[52.26, 9.75], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

# Save as HTML and show in Browser
#https://ipywidgets.readthedocs.io/en/latest/embedding.html#python-interface

In [273]:
from ipywidgets.embed import embed_minimal_html #, dependency_state

embed_minimal_html('ADFC-Map.html', views=[m], title='ADFC Hem/Pat Karte')

# Start browser
!c:\DataADFC\1_adfc-github\adfc-hemmingen-pattensen.github.io\MaengelKarte\ADFC-Map.html

## Remove 2x zoom_offset in *.html to get map back again. 2023-04-15-Sat_20'38

In [274]:
m.save('ADFC-Map Test.html', title='ADFC Hem/Pat Test')